## Direct solve

In [ ]:
using OptimalControl

# Parameters
Cd = 310
Tmax = 3.5
β = 500
b = 2
N = 100
t0 = 0
r0 = 1
v0 = 0
vmax = 0.1
m0 = 1
mf = 0.6

ocp = @def begin

    variable: tf
    time: t ∈ [ t0, tf ]
    state: x[3]
    control: u
    
    r = x₁
    v = x₂
    m = x₃
   
    x(t0) == [ r0, v0, m0 ]
    0  ≤ u(t) ≤ 1
    0  ≤ r(t)            => eq1
    0  ≤ v(t) ≤ vmax     => eq2
    mf ≤ m(t) ≤ m0       => eq3
 
    r(tf) -> max
    
end

function F0(x)
    r, v, m = x
    D = Cd * v^2 * exp(-β*(r - 1))
    F = [ v, -D/m - 1/r^2, 0 ]
    return F
end

function F1(x)
    r, v, m = x
    F = [ 0, Tmax/m, -b*Tmax ]
    return F
end

f(x, u) = F0(x) + u*F1(x)

@def! ocp ẋ(t) == f(x(t), u(t))

sol = solve(ocp)
plot(sol)